Get environment variable
===

In [1]:
import os

NAME = os.environ.get("POSTGRES_DB")
USER = os.environ.get("POSTGRES_USER")
PASSWORD = os.environ.get("POSTGRES_PASSWORD")
HOST = os.environ.get("POSTGRES_HOST")
PORT = os.environ.get("POSTGRES_PORT")

In [2]:
NAME, USER, PASSWORD, HOST, PORT

('celery_db', 'user', 'password', 'db', '5432')

> set variable

In [3]:
os.environ.setdefault("my_key", "my_value")

'my_value'

> get variable

In [4]:
os.environ.get('my_key')

'my_value'

***

Sqlalchemy
===

In [5]:
from sqlalchemy import create_engine
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship

DATABASE_CONNECTION_URI = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'
engine = create_engine(DATABASE_CONNECTION_URI)

In [6]:
Base = declarative_base()


class User(Base):
    __tablename__ = 'my_user'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", backref="my_user",
                             order_by="Address.id")


class Address(Base):
    __tablename__ = 'address'

    id = Column(Integer, primary_key=True)
    user_id = Column(ForeignKey('my_user.id'))
    email_address = Column(String)


Base.metadata.create_all(engine)

Peewee ORM
===

In [7]:
import peewee

Redis
===

In [8]:
from redis import Redis
r = Redis(host='redis', port=6379)

In [9]:
r.get('name')

b'bogdan'

Mongodb
===

In [10]:
import pymongo

Celery
===

In [11]:
import birthdays

In [12]:
hw = birthdays.hello_world.delay()

In [13]:
hw.get()

'hello world matherfucker!'

In [14]:
ad = birthdays.add_digit.delay(4, 5)

In [15]:
ad.get()

9

In [16]:
sr = birthdays.send_row4pause.delay("Bogan", 25)

In [17]:
sr.get()

'hello Bogan'